# Prototype Development

In [49]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from utils import *
from plots import *

data_path = Path("..") / "data"
counts_path = data_path / "Pedestrian_Counting_System___2009_to_Present__counts_per_hour_.csv"
sensor_path = data_path / "Pedestrian_Counting_System_-_Sensor_Locations.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First we'll prep some dataset metadata that we'll need: 

In [8]:
foot_df = load_and_clean_pedestrian_data(counts_path, lat_long_path=sensor_path)

In [14]:
import calendar

MONTHS = list(calendar.month_name)[1:]
WEEKDAYS = list(calendar.day_name)
YEARS = sorted(foot_df["Year"].unique())
SENSORS = sorted(foot_df["Sensor_Name"].unique())
NUM_SNESORS = len(SENSORS)

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [15]:
import ipywidgets as widgets

In [16]:
widgets.Dropdown(options=SENSORS)

Dropdown(options=('231 Bourke St', 'Alfred Place', 'Australia on Collins', 'Birrarung Marr', 'Bourke St - Spen…

In [17]:
widgets.Dropdown(options=YEARS)

Dropdown(options=(2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), value=2009)

In [18]:
first_year = min(YEARS)
last_year = max(YEARS)
widgets.IntRangeSlider(
    value=[first_year, last_year],
    min=first_year,
    max=last_year,
)

IntRangeSlider(value=(2009, 2020), max=2020, min=2009)

## Prototype

In [186]:
# TODO
# * values of control widgets should be constrained by current value of other widgets
#   - change update_inputs func to use the change param
#   - need to sort the new month order according to months

from functools import partial

def prototype(df):
    # inputs
    year_input = widgets.Dropdown(options=YEARS, value=YEARS[-1])
    sensor_input = widgets.SelectMultiple(options=SENSORS, value=[])
    month_input = widgets.Dropdown(options=MONTHS, value=None)

    def update_inputs(change):
        print(args)
        filtered_df = filter_foot_df(
            df,
            year=year_input.value,
            month=month_input.value,
            sensor=sensor_input.value,
        )
        return
        if "month" in args:
            month_input.options = sorted(filtered_df["Month"].unique())
            if month_input.value not in month_input.options:
                month_input.value = month_input.options[0]
        if "year" in args:
            year_input.options = sorted(filtered_df["Year"].unique())
            if year_input.value not in year_input.options:
                year_input.value = year_input.options[0]
        if "sensor" in args:
            sensor_input.options = sorted(filtered_df["Sensor_Name"].unique())
            sensor_input.value = [sensor for sensor in sensor.value if sensor in sensor_input.options]
            
    # outputs
    month_counts_output = widgets.interactive_output(
        display_output(functools.partial(plot_month_counts, df)),
        {"sensor": sensor_input, "year": year_input},
    )
    sensor_counts_output = widgets.interactive_output(
        display_output(functools.partial(plot_sensor_counts, df)),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )
    sensors_output = widgets.interactive_output(
        display_output(functools.partial(plot_sensors, df, width=1500)),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )
    sensors_map_output = widgets.interactive_output(
        display_output(functools.partial(plot_scatter_map, df, height=900)),
        {"sensor": sensor_input, "year": year_input, "month": month_input},
    )
    
    year_input.observe(update_inputs, "value")
    month_input.observe(update_inputs, "value")
    sensor_input.observe(update_inputs, "value")

    # layout
    label_layout = {"width": "5em"}
    row1 = widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.HTML("<H1>Melbourne City Council Pedestrian Traffic</h1>"),
                    widgets.HBox(
                        [widgets.HTML("<b>Year</b>", layout=label_layout), year_input]
                    ),
                    widgets.HBox(
                        [widgets.HTML("<b>Month</b>", layout=label_layout), month_input]
                    ),
                    widgets.HBox(
                        [
                            widgets.HTML("<b>Sensors</b>", layout=label_layout),
                            sensor_input,
                        ]
                    ),
                ]
            ),
            month_counts_output,
        ]
    )
    row2 = widgets.HBox([sensors_map_output, sensor_counts_output])
    row3 = widgets.HBox([sensors_output])
    layout = widgets.VBox([row1, row2, row3])
    return layout


prototype(foot_df)

In [ ]:
def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

In [ ]:
    row1 = widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.HTML("<H1>Melbourne City Council Pedestrian Traffic</h1>"),
                    widgets.HBox(
                        [widgets.HTML("<b>Year</b>", layout=label_layout), year_input]
                    ),
                    widgets.HBox(
                        [widgets.HTML("<b>Month</b>", layout=label_layout), month_input]
                    ),
                    widgets.HBox(
                        [
                            widgets.HTML("<b>Sensors</b>", layout=label_layout),
                            sensor_input,
                        ]
                    ),
                ]
            ),
            month_counts_output,
        ]
    )

In [156]:
widgets.?

Init signature: widgets.Label(*args, **kwargs)
Docstring:     
Label widget.

It also renders math inside the string `value` as Latex (requires $ $ or
$$ $$ and similar latex tags).
Init docstring: Public constructor
File:           ~/.pyenv/versions/3.8.3/envs/footviz/lib/python3.8/site-packages/ipywidgets/widgets/widget_string.py
Type:           MetaHasTraits
Subclasses:     


In [158]:
b = widgets.HTML("Foo")
b.style.keys

['_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'description_width']

## Comparing Sensors by Year

In [72]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label

In [30]:
is_str_or_non_empty([1,2,4])

True

In [41]:
import numpy as np
import numbers

In [43]:
isinstance(np.array([1])[0], numbers.Number)

True

In [63]:
isinstance(np.array([1]), collections.abc.MutableSequence)

False